Dependencies

In [1]:
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
import os
import time

Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
# an image and mp holistic model::returns landmarks as results
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # converting color from bgr to rgb
    image.flags.writeable = False #saves memory during processing   
    results = model.process(image)#make prediction poses
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22, 76), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(121,44, 250), thickness=1, circle_radius=1),)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117, 66), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(245,66, 230), thickness=1, circle_radius=1),)

Extracting Landmark values

In [5]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])


In [6]:
#Folders for collections
DATA_PATH = os.path.join("collection")

#Actions that we try to detect
actions = np.array(['YES', 'iloveyou', 'HELLO'])
#Thirty videos worth of data
no_sequences = 30
#Videos are going to be 30 frames in length
sequence_length = 30

In [14]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

Collecting keypoint values for Training and Testing

In [15]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                #read frame
                ret, frame = cap.read()
                #make detections-cc
                image , results = mediapipe_detection(frame, holistic)
                #drawing real time landmarks
                draw_styled_landmarks(image, results)
                #Collection Logic
                if frame_num==0:
                    #image, text, position, font type, font size, color, line width, line type
                    cv2.putText(image, 'Starting Collection', (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15 ,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 4, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15 ,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)            
                #New export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints, allow_pickle=True, fix_imports=True)
                #display frame
                cv2.imshow("OpenCV Feed", image)
                
                #close the display
                if cv2.waitKey(10) & 0xFF ==ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

Preprocess Data and Create Labels

In [7]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [8]:
label_map = {label : num for num, label in enumerate(actions)}

In [9]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])


In [10]:
X = np.array(sequences)
Y = to_categorical(labels).astype(int)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.05)

Building and Training a LSTM(Long Short Term Memory) Neural Network

In [11]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard 

In [12]:
log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [13]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(actions.shape[0], activation='softmax'))


In [14]:
res = [0.7, 0.2, 0.1]
actions[np.argmax(res)]

'YES'

In [15]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) #categorical used for multi-class classification

In [37]:
model.fit(X_train, Y_train, epochs=2000, callbacks=tb_callback)

Epoch 1/2000
3/3 [==============================] - 3s 38ms/step - loss: 1.1012 - categorical_accuracy: 0.2353
Epoch 2/2000
3/3 [==============================] - 0s 35ms/step - loss: 1.0933 - categorical_accuracy: 0.4000
Epoch 3/2000
3/3 [==============================] - 0s 35ms/step - loss: 1.0909 - categorical_accuracy: 0.3529
Epoch 4/2000
3/3 [==============================] - 0s 35ms/step - loss: 1.0766 - categorical_accuracy: 0.4353
Epoch 5/2000
3/3 [==============================] - 0s 36ms/step - loss: 1.0538 - categorical_accuracy: 0.4941
Epoch 6/2000
3/3 [==============================] - 0s 38ms/step - loss: 1.0326 - categorical_accuracy: 0.5059
Epoch 7/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.9945 - categorical_accuracy: 0.4706
Epoch 8/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.9644 - categorical_accuracy: 0.6000
Epoch 9/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.9225 - categorical_accuracy: 0.4588
E

In [38]:
res = model.predict(X_test)

1/1 [==============================] - 0s 331ms/step


In [39]:
actions[np.argmax(res[0])]

'iloveyou'

Saving the Model

In [40]:
model.save('demo6.h5')

Loading our model


In [16]:
model.load_weights('demo6.h5')

In [17]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [18]:
yhat = model.predict(X_train)

3/3 [==============================] - 1s 26ms/step


In [19]:
ytrue = np.argmax(Y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [20]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[56,  0],
        [ 0, 29]],

       [[56,  0],
        [ 0, 29]],

       [[58,  0],
        [ 0, 27]]], dtype=int64)

In [21]:
accuracy_score(ytrue, yhat)# checking accuracy of the model

1.0

TESTING SHEEESH


In [22]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [23]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        draw_styled_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # image = prob_viz(res, actions, image, colors)
             
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [29]:
cap.release()
cv2.destroyAllWindows()
